In [92]:
# Import libraries
import xml.etree.cElementTree as et
import pandas as pd

In [93]:
# Define function to get values if they exist
def getvalueofnode(node):
    return node.text if node is not None else None

In [94]:
# Load data
parsed_xml = et.parse("dec_data.xml")

In [95]:
# Load points and wins into DataFrames
dfcols = ['ballot', 'recipient', 'score']
# Define DataFrames for points and wins
points = pd.DataFrame(columns=dfcols)
wins = pd.DataFrame(columns=dfcols)

# For each 'BALLOT_SCORE', add to points or wins depending on what it is
for node in parsed_xml.getroot().findall("BALLOT_SCORE"):
    ballot = node.find('BALLOT')
    recipient = node.find('RECIPIENT')
    score = node.find('SCORE')

    if node.find('SCORE_ID').text == 'POINTS':
      points = points.append(
        pd.Series([getvalueofnode(ballot), getvalueofnode(recipient), float(getvalueofnode(score))], index=dfcols),
        ignore_index=True)
    else:
      wins = wins.append(
        pd.Series([getvalueofnode(ballot), getvalueofnode(recipient), getvalueofnode(score)], index=dfcols),
        ignore_index=True)

In [96]:
points.head()

,ballot,recipient,score
0,12853384,756282,74.0
1,12853384,600714,79.0
2,12853384,604522,79.0
3,12853385,756147,79.0
4,12853385,756119,76.0


In [97]:
# Load student information
students = pd.DataFrame(columns=['id', 'school', 'team', 'first', 'last'])

# For each 'ENTRY_STUDENT', add to students DataFrame
for node in parsed_xml.getroot().findall("ENTRY_STUDENT"):
  id = node.find('ID')
  school = node.find('SCHOOL')
  team = node.find('ENTRY')
  first = node.find('FIRST')
  last = node.find('LAST')

  students = students.append(pd.Series([getvalueofnode(id), getvalueofnode(school), 
                                        getvalueofnode(team), getvalueofnode(first), 
                                        getvalueofnode(last)], index=students.columns), ignore_index=True)
  
# Remove grade level from student name and add it to grade column
students['grade'] = students['first'].str.extract('(\d+)')
students['first'] = students['first'].str.replace('\d+', '')

In [98]:
students.head()

,id,school,team,first,last,grade
0,755264,244063,2140762,Paige,Bringas,6
1,672912,244063,2140762,Joy,Cho,6
2,755266,244063,2140762,Rebekah,Kim,6
3,598796,244063,2140748,Elizabeth,Choe,6
4,755253,244063,2140748,Zyaire,Eastland,6


In [99]:
# Label points with student names
points_with_names = points.merge(students, left_on='recipient', right_on='id')

In [100]:
points_with_names

,ballot,recipient,score,id,school,team,first,last,grade
0,12853384,756282,74.0,756282,244518,2146448,David,Lu,21
1,12860681,756282,74.0,756282,244518,2146448,David,Lu,21
2,12868603,756282,79.0,756282,244518,2146448,David,Lu,21
3,12872761,756282,71.0,756282,244518,2146448,David,Lu,21
4,12853384,600714,79.0,600714,244518,2146448,Rachel,Nazareth,21
...,...,...,...,...,...,...,...,...,...
1407,12872837,755431,80.0,755431,242458,2141361,Siddhanth,Kumar,8
1408,12853401,755426,80.0,755426,242458,2141361,Ayushi,Kadakia,8
1409,12860640,755426,76.0,755426,242458,2141361,Ayushi,Kadakia,8
1410,12868644,755426,82.0,755426,242458,2141361,Ayushi,Kadakia,8


In [101]:
student_performance = points_with_names.groupby('recipient').agg({'score': ('max', 'min')})

In [102]:
student_performance.head()

score      
            max   min
recipient            
459193     81.0  75.0
459198     78.0  75.0
459581     78.0  75.0
460423     82.0  74.0
460427     84.0  77.0

In [103]:
student_performance['score', 'range'] = student_performance['score', 'max'] - student_performance['score', 'min']

In [104]:
high_range = student_performance[student_performance['score', 'range'] >= 10]
print(high_range.shape[0] / student_performance.shape[0])
high_range.head()

0.1954674220963173


score            
            max   min range
recipient                  
599062     84.0  74.0  10.0
599826     84.0  74.0  10.0
600691     85.0  70.0  15.0
600692     86.0  72.0  14.0
600694     83.0  72.0  11.0